In [6]:
import os
import re
import json
import time
import openai
import requests
import numpy as np
import pandas as pd
from lxml import etree
from openai import OpenAI
from bs4 import BeautifulSoup
# from sklearn.neighbors import NearestNeighbors

In [10]:
import os
os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

In [1]:
import openai
from openai import OpenAI

In [8]:
api_key = ""
api_base = ""
client = OpenAI(api_key=api_key, base_url=api_base)

In [9]:
dialogue_model = "gpt-4o-mini" # "gpt-3.5-turbo"
embed_model = "text-embedding-3-small"

# 初始化两个Database
预想应该有两个数据库：    
一个数据库是完全体，包含已经计算过的嵌入，相似indices，距离了；      
另一个是半成品，用于储存爬取下来的信息，不包含嵌入和距离之类的。注意，数据库应该在每次与完全体数据库合并之后进行清除

In [ ]:
# 初始化完全体向量数据库
# ['名称', '作者', '日期', '摘要', '摘要总结', '摘要总结的嵌入']
complete_columns = ['title', 'Author', 'Date', 'Abstract', 'Abstract_Summaried', ''Abstract_Summaried embeddings'] 
completeDatabase = pd.DataFrame(columns=complete_columns)
completeDatabase.head(5)

In [ ]:
# 初始化半成品数据库
# ['名称', '作者', '日期', '摘要']
semicomplete_columns = ['title', 'Author', 'Date', 'Abstract'] 
semicompleteDatabase = pd.DataFrame(columns=semicomplete_columns)
semicompleteDatabase.head(5)

# 使用函数提取用户输入中的关键信息

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_query_info",
            "description": "Extracts query information from the user input including whether to initiate crawling, the domain that users need to query, and the specific content being queried.",
            "parameters": {
                "type": "object",
                "properties": {
                    "crawl": {
                        "type": "boolean",
                        "description": "Whether the user explicitly requests to perform crawling (e.g., by using phrases like '需要爬取')."
                    },
                    "domain": {
                        "type": "string",
                        "description": "The domain that users need to query (e.g., literature, computer science, data science, etc.)"
                    },
                    "content": {
                        "type": "string",
                        "description": "The specific content or topic within the domain being queried (e.g., 红楼梦, 位置编码, positional encoding, etc.)."
                    },
                    "major": {
                        "type": "boolean",
                        "description": "Is the field to be inquired related to science and engineering?"
                    }
                },
                "required": ["crawl", "domain", "content", "major"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "summary_abstract",
            "description": "Summarize the abstract of the designated article and limit it to 60 words or less.",
            "parameters": {
                "type": "object",
                "properties": {
                    "results": {
                        "type": "string",
                        "description": "Summary of the article abstract."
                    }
                },
                "required": ["results"]
            }
        }
    }
]

In [11]:
# test
messages = [{"role": "user", "content": "我需要爬取关于位置编码的有关内容"}]
response = client.chat.completions.create(model=dialogue_model, messages=messages,
                                          tools=tools, tool_choice={"type": "function", "function": {"name": "extract_query_info"}})
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WVKd0H8gEojfJl6EOE09AWoZ', function=Function(arguments='{"content":"位置编码","crawl":true,"domain":"计算机科学","major":true}', name='extract_query_info'), type='function')])


In [12]:
response

ChatCompletion(id='chatcmpl-9nOjATWRlryha4haqS1UVWzsewrtA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WVKd0H8gEojfJl6EOE09AWoZ', function=Function(arguments='{"content":"位置编码","crawl":true,"domain":"计算机科学","major":true}', name='extract_query_info'), type='function')]))], created=1721560152, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_8b761cb050', usage=CompletionUsage(completion_tokens=20, prompt_tokens=225, total_tokens=245))

In [ ]:
# test
print(
    response.choices[0].message.tool_calls, "\n", \
    response.choices[0].message.tool_calls[0].function.arguments, "\n", \
    json.loads(response.choices[0].message.tool_calls[0].function.arguments)["domain"]
)

# 向量化用户请求

In [ ]:
def get_query_embedding(request):
    """
    request是用户的输入，是一个string
    """
    response = client.embeddings.create(input=request, model=embed_model)
    return response.data[0].embedding

In [ ]:
# test
request_embeddings = get_query_embedding("cat is not dag")
len(request_embeddings)

# 爬虫
已经有了json.loads(response.choices[0].message.tool_calls[0].function.arguments)，类似：{"content":"位置编码","crawl":true,"domain":"科学与工程","major":true}


In [ ]:
# test
print(json.loads(response.choices[0].message.tool_calls[0].function.arguments), "\n", type(json.loads(response.choices[0].message.tool_calls[0].function.arguments)))

In [3]:
base_urls = {
    "ACM": "",
    "知网空间": 'http://search.cnki.com.cn/Search/ListResult'
}

### 知网 

In [11]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0
def get_page_text(url, search_word, page_num):
    data = {
        'searchType': 'MulityTermsSearch', 'ArticleType': '', 'ReSearch': '', 'ParamIsNullOrEmpty': 'false', 'Islegal': 'false',
        'Content': search_word,
        'Theme': '', 
        'Title': '', 'KeyWd': '', 'Author': '', 'SearchFund': '', 'Originate': '', 'Summary': '', 'PublishTimeBegin': '', 'PublishTimeEnd': '', 
        'MapNumber': '', 'Name': '', 'Issn': '', 'Cn': '', 'Unit': '', 'Public': '', 'Boss': '', 'FirstBoss': '', 'Catalog': '', 'Reference': '', 'Speciality': '', 
        'Type': '', 'Subject': '', 'SpecialityCode': '', 'UnitCode': '', 'Year': '', 'AcefuthorFilter': '', 'BossCode': '', 'Fund': '', 'Level': '', 'Elite': '', 
        'Organization': '', 'Order': '1',
        'Page': str(page_num),
        'PageIndex': '', 'ExcludeField': '', 'ZtCode': '', 'Smarts': '',
    }
    response = requests.post(url=url, headers=headers, data=data)
    page_text = response.text
    return page_text

def list_to_str(my_list):
    my_str = "".join(my_list)
    return my_str

def get_abstract(url):
    response = requests.get(url=url, headers=headers)
    page_text = response.text
    tree = etree.HTML(page_text)
    abstract = tree.xpath('//div[@class="xx_font"]//text()')
    return abstract

def parse_page_text(page_text, top_n):
    tree = etree.HTML(page_text)
    item_list = tree.xpath('//div[@class="list-item"]')
    # print(type(item_list), len(item_list)) # list类型，长度为20
    page_info = []
    for item in item_list[0:top_n]: # 只解析前top_n篇
        # 标题
        title = list_to_str(item.xpath(
            './p[@class="tit clearfix"]/a[@class="left"]/@title'))
        # 链接
        link = 'https:' +\
            list_to_str(item.xpath(
                './p[@class="tit clearfix"]/a[@class="left"]/@href'))
        # 作者
        author = list_to_str(item.xpath(
            './p[@class="source"]/span[1]/@title'))
        # 出版日期
        date = list_to_str(item.xpath(
            './p[@class="source"]/span[last()-1]/text() | ./p[@class="source"]/a[2]/span[1]/text() '))
        # 关键词
        keywords = list_to_str(item.xpath(
            './div[@class="info"]/p[@class="info_left left"]/a[1]/@data-key'))
        # 摘要
        abstract = list_to_str(get_abstract(url=link))
        # 文献来源
        paper_source = list_to_str(item.xpath(
            './p[@class="source"]/span[last()-2]/text() | ./p[@class="source"]/a[1]/span[1]/text() '))
        # 文献类型
        paper_type = list_to_str(item.xpath(
            './p[@class="source"]/span[last()]/text()'))
        # 下载量
        download = list_to_str(item.xpath(
            './div[@class="info"]/p[@class="info_right right"]/span[@class="time1"]/text()'))
        # 被引量
        refer = list_to_str(item.xpath(
            './div[@class="info"]/p[@class="info_right right"]/span[@class="time2"]/text()'))

        item_info = [i.strip() for i in [title, author, paper_source, paper_type, date, abstract, keywords, download, refer, link]]
        page_info.append(item_info)
        # print(page_info)
    return page_info

In [14]:
# test
page_text = get_page_text(base_urls["知网空间"], "位置编码", 1)
page_info = parse_page_text(page_text, 30)

In [ ]:
# test
print(len(page_info), page_info[20])

In [ ]:
# test
for paper_info in page_info:
    paper_info_selected = [paper_info[i] for i in [0, 1, 4, 5]] # title, author, date, abstract
    new_row = pd.DataFrame([dict(zip(semicomplete_columns, paper_info_selected))])
    semicompleteDatabase = pd.concat([semicompleteDatabase, new_row], ignore_index=True)
    
semicompleteDatabase.head(5)

In [ ]:
for abstract in semicompleteDatabase["Abstract"]:
    print(abstract)
    break

# 将摘要传给模型，让模型总结一下

In [ ]:
# test
for abstract in semicompleteDatabase["Abstract"]:
    messages = [{"role": "system", "content": "你是一位擅长文字总结的助手。我将发送给你一篇文章的摘要，你需要总结它，并限制在60个字以内。"},
                {"role": "user", "content": "你需要总结的摘要如下："+abstract}]
    response = client.chat.completions.create(model=dialogue_model, messages=messages, 
                                              tools=tools, tool_choice={"type": "function", "function": {"name": "summary_abstract"}})
    print(response.choices[0].message)
    break

In [ ]:
print(response.choices[0].message.tool_calls[0].function.arguments, "\n", json.loads(response.choices[0].message.tool_calls[0].function.arguments)['results'])

# 处理domain和content中存在空格的现象

In [ ]:
def replace_spaces_with_plus(input_string):
    if ' ' in input_string:
        return input_string.replace(' ', '+')
    else:
        return input_string

### ACM

In [ ]:
def Crawler_ACM(domain, content, num_pages):
    entries = []
    i=0
    while i < num_pages: # 爬取几页
        url = "https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=AllField&text1="+domain+"&field2=AllField&text2="+content+"&startPage="+str(i)+"&pageSize=20"
        print(url)
        content = requests.get(url).text
        page = BeautifulSoup(content, 'lxml')
        for entry in page.find_all("div", attrs={"class": "issue-item__content"}):
            entries.append(entry)

        i+=1
    return entries

In [ ]:
domain = "positional+encoding"
content = "CNN"
num_pages = 1

results = Crawler_ACM(domain, content, num_pages)

In [ ]:
len(results)

In [ ]:
def Crawler_ACM(domain, content, num_pages):
    results = []
    i=0
    while i < num_pages: # 爬取几页
        url = "https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=AllField&text1="+domain+"&field2=AllField&text2="+content+"&startPage="+str(i)+"&pageSize=20"
        print(url)
        content = requests.get(url).text
        page = BeautifulSoup(content, 'lxml')
        for entry in page.find_all("div", attrs={"class": "issue-item__content"}):
            title = entry.find('h5', attrs={'class': 'issue-item__title'})
            author = entry.find('ul', attrs={'class': 'rlist--inline'})
            years = entry.find('span', attrs={'class': 'dot-separator'})
            
            print(type(title), type(author), type(years), title, author, years)
            break

            # 暂时存储中间结果
            results.append({
                "title": title.text.replace('[PDF]', '').strip() if title else '',
                'authors': author.text.replace('\n', '').strip() if author else '',
                'years': years.text.split(',')[0].split(' ')[1].strip() if years else '',
                'abstract': ''  # 先初始化为空字符串
            })
            
        abstract_entries = page.find_all("div", attrs={"class": "accordion__content"})
        for i, abst_entry in enumerate(abstract_entries):
            abst = abst_entry.find('div', attrs={'class': 'abstract-text'})
            if i < len(results):
                results[i]['abstract'] = abst.text.replace('\n', '').strip() if abst else ''
                
        # Add a sleep of 1 seconds between requests
        time.sleep(3)
        
        i=i+1
        
    return results

In [ ]:
domain = "positional+encoding"
content = "CNN"
num_pages = 1

results = Crawler_ACM(domain, content, num_pages)

In [ ]:
results

In [ ]:
df = pd.DataFrame(results)

In [ ]:
for paper_info in results:
    paper_info_selected = [paper_info[i] for i in ["title", 'authors', 'years', 'abstract']] # title, author, date, abstract
    new_row = pd.DataFrame([dict(zip(semicomplete_columns, paper_info_selected))])
    semicompleteDatabase = pd.concat([semicompleteDatabase, new_row], ignore_index=True)
    
semicompleteDatabase.head(5)

## 汇总爬虫

In [ ]:
def crawler(request_vector, params):
    """
    url：用户需要爬取网站的url，例如：
    params：第一次将用户请求输入模型的输出中的字典。与url结合。例如：https://httpbin.org/get?key1=value1&key2=value2
    """
    crawl = params['crawl'] # 用户是否指明需要爬取
    domain = params['domain'] # 领域
    content = params['content'] # 领域内具体内容
    major = params['major'] # 是否为理工科

    urlparams = {'domain': domain, 'content': content}

    if crawl: # 如果指定需要爬取
        if major: # 如果是理工科，则使用ACM查询
            pass
        else: # 文科用知网查询
            page_info = parse_page_text(get_page_text(base_urls["知网空间"], "domain + content", 1), 5) # 只爬取前5篇
            for paper_info in page_info:
                paper_info_selected = [paper_info[i] for i in [0, 1, 4, 5]] # title, author, date, abstract
                new_row = pd.DataFrame([dict(zip(semicomplete_columns, paper_info_selected))])
                semicompleteDatabase = pd.concat([semicompleteDatabase, new_row], ignore_index=True)
    else: # 如果没有指定，则调用find_similarParagraphs()计算用户请求与completeDatabase中Abstract列的相似度
        similarParagraphs = find_similarParagraphs(request_vector)
        if len(similarParagraphs) == 0:
            if major: # 如果是理工科，则使用ACM查询
                pass
            else: # 文科用知网查询
                page_info = parse_page_text(get_page_text(base_urls["知网空间"], "domain + content", 1), 5) # 只爬取前5篇
                for paper_info in page_info:
                    paper_info_selected = [paper_info[i] for i in [0, 1, 4, 5]] # title, author, date, abstract
                    new_row = pd.DataFrame([dict(zip(semicomplete_columns, paper_info_selected))])
                    semicompleteDatabase = pd.concat([semicompleteDatabase, new_row], ignore_index=True)
    
    return semicompleteDatabase

# 将semicompleteDatabase中的Abstract列使用嵌入模型进行嵌入

In [ ]:
def create_embeddings(text):
    # Create embeddings for each document chunk
    embeddings = openai.embeddings.create(input = text, model=embed_model).data[0].embedding
    return embeddings

embeddings = []
for Abstract in flattened_df['Abstract']:
    embeddings.append(create_embeddings(Abstract))

# store the embeddings in the dataframe
semicompleteDatabase['Abstract embeddings'] = embeddings

# 整合semicompleteDatabase和completeDatabase
completeDatabase = pd.concat([completeDatabase, semicompleteDatabase])

# 计算request和completeDatabase中Abstract列中每个元素的相似度

In [ ]:
def find_similarParagraphs(request_vector, threshold=0.6, top_n=5):
    paragraph_embeddings = completeDatabase['Paragraph embeddings'].to_list()
    nbrs = NearestNeighbors(n_neighbors=top_n, algorithm='ball_tree').fit(paragraph_embeddings) # 这里每次调用这个函数都会计算一次，maybe会增加时间成本
    
    distances, indices = nbrs.kneighbors([request_vector])
    
    similarParagraphs = []
    for index, distance in zip(indices[0], distances[0]):
        if distance < threshold:
            similarParagraphs.append(completeDatabase['Abstract'].iloc[index])

    if len(similarParagraphs) == 0:
        return "需要使用爬虫爬其，请等待！" #############################################################################################################
    else: 
        return similarParagraphs

# 将返回的similarParagraphs（list）作为prompt一起传入模型中

In [ ]:
def chatbot(request):

    request_vector = get_query_embedding(request)

    similarParagraphs = find_similarParagraphs(request_vector)
    
    total_content = 
    
    # create a message object
    messages=[
        {"role": "system", "content": "You are an AI assiatant that helps with some questions."},
        {"role": "user", "content": total_content}
    ]

    # use chat completion to generate a response
    response = openai.chat.completions.create(
        model=dialogue_model,
        messages=messages,
        temperature=0.7,
        max_tokens=800,
    )

    return response.choices[0].message

In [ ]:
chatbot(user_input)